In [28]:
#IMPORTING LIBRARIES:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

pd.set_option('display.max_columns', 100)
sns.set_style("darkgrid")

from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import re

from sklearn.feature_extraction.text import CountVectorizer

# IF YOU ARE MISSING "WordCloud":
# TRY INSTALLING VIA TERMINAL LIKE THIS: /anaconda3/bin/python -m pip install wordcloud
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from PIL import Image
# IF YOU ARE MISSING "wordninja":
# TRY INSTALLING VIA TERMINAL LIKE THIS: pip install wordninja
import wordninja


%matplotlib inline
import scattertext as st
import re, io
from pprint import pprint
from scipy.stats import rankdata, hmean, norm
import spacy
import os, pkgutil, json, urllib
from urllib.request import urlopen
from IPython.display import IFrame
from IPython.display import display, HTML
from scattertext import CorpusFromPandas, produce_scattertext_explorer
display(HTML("<style>.container { width:98% !important; }</style>"))

In [4]:
# #IMPORTING CSV FILES from what was extracteed recently and data retreived from kaggle
depression = pd.read_csv(r"D:\Pranav_Files\Northwestern University\Classes\MSDS_498_Capstone_Project\Data and Git\Capstone_Project\data\vics_diary_copy.csv")
# suicide_watch = pd.read_csv('../data/suicide_watch.csv')
# depression_old = pd.read_csv('../data/OLD_DATA/depression.csv')
# suicide_old =pd.read_csv('../data/OLD_DATA/suicide_watch.csv')
# new_data=pd.read_csv(r"D:\Pranav_Files\Northwestern University\Classes\MSDS_498_Capstone_Project\Data and Git\Capstone_Project\data\vics_diary_copy.csv")

In [5]:
# new_data=new_data.drop(columns="Unnamed: 0")
new_data= pd.DataFrame()
new_data["text"]= depression["journ_entry"]
new_data["class"]=depression["vic_detail"]

In [6]:
pd.set_option('display.max_columns', 500)
new_data.head()

,text,class
0,"""I don 't want other kids to feel like freaks ...",depression
1,"""If I'm brain damaged,""\nit read, ""I don't wan...",high-suicidal
2,NaN,NaN
3,Winter Mind - It's a shadow that sweeps over y...,suicidal
4,NaN,NaN


In [7]:
print(new_data["text"][18])
len(new_data["text"][18])

"I woke up early with my heart pounding,"
... it was seven in the morning and somehow the cortisol or adrenaline
or whatever chemical it is forced me out of bed. I was thinking
about all the untouched homework. Kind of shocked me how much
I hadn't done and how much I was supposed to do. I tried going back
to sleep but I just couldn't stay still. I kept thrashing around like I was
having a nightmare. It was that stage between a panic attack and that
nauseous fear clawing at your stomach . ... Dammit I wish I would
stop feeling sick. Jeez, why do I always do this? I'm an idiot! I had
the whole fucking holidays! I have so much work to do, it's insane.
And the year has barely begun.


687

In [8]:
print("There are ", new_data["text"].isnull().sum(), " NAN Values")
## Inspecting data
new_data[new_data["text"].isnull()].head()

There are  2  NAN Values


,text,class
2,NaN,NaN
4,NaN,NaN


In [9]:
drop_index=new_data[new_data["text"].isnull()].index
new_data.drop(index=drop_index,inplace=True)

In [10]:
## Checking
new_data[new_data["text"].isnull()]


,text,class


In [11]:
combined_data=new_data.reset_index(drop=True)

In [12]:
####BAaCKING UP combined_data IN A CSV FILE 
# combined_data.to_csv('../data/new_suicide_data.csv', index = False)

In [2]:
# combined_data=pd.read_csv('../data/new_suicide_data.csv')

### 2.2 Pre-processing

In [13]:

def processing_text(series_to_process):
    new_list = []
    tokenizer_regex = RegexpTokenizer(r'(\w+)')
    lemmatizer = WordNetLemmatizer()
    for i in range(len(series_to_process)):
        #TOKENISED ITEM(LONG STRING) IN A LIST
        dirty_string = (series_to_process)[i].lower()
        words_only = tokenizer_regex.tokenize(dirty_string) #WORDS_ONLY IS A LIST THAT DOESN'T HAVE PUNCTUATION
        replacing_ele=[x.replace('\n','') for x in words_only]
        replacing_ele1=[x.replace('\r','') for x in words_only]
        #LEMMATISE THE ITEMS IN WORDS_ONLY
        words_only_lem = [lemmatizer.lemmatize(i) for i in replacing_ele1]
        #REMOVING STOP WORDS FROM THE LEMMATIZED LIST
#         words_without_stop = [i for i in words_only_lem if i not in stopwords.words("english")]
        #RETURN SEPERATED WORDS INTO LONG STRING
        long_string_clean = " ".join(word for word in words_only_lem)
        new_list.append(long_string_clean)
    return new_list

In [14]:
##Cleaning and processing self text and title and creating seperate coumns
##CHECKING TO SEE NEW COLUMNS WERE ADDED
combined_data["text_clean"] = processing_text(combined_data["text"])

In [16]:
combined_data

,text,class,text_clean
0,"""I don 't want other kids to feel like freaks ...",depression,i don t want other kid to feel like freak or t...
1,"""If I'm brain damaged,""\nit read, ""I don't wan...",high-suicidal,if i m brain damaged it read i don t want to b...
2,Winter Mind - It's a shadow that sweeps over y...,suicidal,winter mind it s a shadow that sweep over you ...
3,"She prayed and prayed,\nThe covers wrapped aro...",suicidal,she prayed and prayed the cover wrapped around...
4,I will be - You lookup.\nStars\n little lights...,high-suicidal,i will be you lookup star little light little ...
5,"""I have so many opportunities,"" ""If some peop...",depression,i have so many opportunity if some people were...
6,"""I've known that I will never have a\ndazzling...",depression,i ve known that i will never have a dazzling l...
7,I think ( and I know it sounds melodramatic ) ...,depression,i think and i know it sound melodramatic that ...
8,"""If I fail these exams, I know that my\n will ...",depression,if i fail these exam i know that my will be ru...
9,"""I'm not that much of an idiot. There are wors...",depression,i m not that much of an idiot there are worse ...


In [17]:
####BAaCKING UP combined_data IN A CSV FILE 
combined_data.to_csv('../data/victoria_diary_clean.csv', index = False)

In [18]:
pd.set_option("display.max_colwidth", 1000)
combined_data.head(-5)

,text,class,text_clean
0,"""I don 't want other kids to feel like freaks or teachers to\nalways assume that the kid at the back of class who never raises their\nhand is just 'shy' . .. when they are really paralyzed with [a] fear and\nhopelessness that they believe no one could ever understand.""",depression,i don t want other kid to feel like freak or teacher to always assume that the kid at the back of class who never raise their hand is just shy when they are really paralyzed with a fear and hopelessness that they believe no one could ever understand
1,"""If I'm brain damaged,""\nit read, ""I don't want to be kept alive. I don 't want to be a\nvegetable: '",high-suicidal,if i m brain damaged it read i don t want to be kept alive i don t want to be a vegetable
2,"Winter Mind - It's a shadow that sweeps over you.\nGrey skies. Charcoal streets coated with grime\nAnd glistening wetness under rusted lampposts.\nRain-splattered glass,the drops slidingd own car windows like tears.\nThe eternal, slow, steady rain falling, on cold days through fog.\nTangled, windswept hair. Stale black coffee.\nDust building on the spines of unopened books.\nThe color draining from floral-patterned sheets and curios\nIn bedrooms like tea-dregs giving way to crumpled black clothes\nAnd silence.\nSplinters of dark aluminum sunrays refracting\nThrough the gaps in drawn curtains.\nTime passing in a colorless blur between day and night.\nThe weeks stretching into an endless winter,:\nKept inside to rot like a dying plant. Eyes opening only to wish,\nTo cling on to the only thing you know; that death will come to you.\nSad songs become a daily commodity.\nNot even talking can expunge this melancholy,\nThere will soon be nothing left to extract.\nSilence.\nThe time has pa...",suicidal,winter mind it s a shadow that sweep over you grey sky charcoal street coated with grime and glistening wetness under rusted lamppost rain splattered glass the drop slidingd own car window like tear the eternal slow steady rain falling on cold day through fog tangled windswept hair stale black coffee dust building on the spine of unopened book the color draining from floral patterned sheet and curio in bedroom like tea dreg giving way to crumpled black clothes and silence splinter of dark aluminum sunray refracting through the gap in drawn curtain time passing in a colorless blur between day and night the week stretching into an endless winter kept inside to rot like a dying plant eye opening only to wish to cling on to the only thing you know that death will come to you sad song become a daily commodity not even talking can expunge this melancholy there will soon be nothing left to extract silence the time ha passed for brutal honesty for there is no energy left to bellow out the ...
3,"She prayed and prayed,\nThe covers wrapped around her,\nListening to the clock tick away her time,\nUntil the day she left the bed,\nAnd left herself behind.",suicidal,she prayed and prayed the cover wrapped around her listening to the clock tick away her time until the day she left the bed and left herself behind
4,"I will be - You lookup.\nStars\n little lights \nLittle places where you\n Cannot live with your \nLungs and your skin But \nwith your heart.\nMy little world will meet\n Yours and yours mine. \nMy organs, though left \nBehind in the earth to become\nThe trees, or to live and\n Be wanted in another's body,\nI will be there as\n Another dot to join the dots,\nSo that when you look up,\nI will be there, and you know \nI will be free.",high-suicidal,i will be you lookup star little light little place where you cannot live with your lung and your skin but with your heart my little world will meet yours and yours mine my organ though left behind in the earth to become the tree or to live and be wanted in another s body i will be there a another dot to join the dot so that when you look up i will be there and you know i will be free
5,"""I have so many opportunities,"" ""If some people

In [19]:
#CHECKING IF OUR FUNCTIONS INTRODUCED MISSING DATA INTO OUR NEW COLS
combined_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56 entries, 0 to 55
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   text        56 non-null     object
 1   class       56 non-null     object
 2   text_clean  56 non-null     object
dtypes: object(3)
memory usage: 1.4+ KB


In [20]:
#DOUBLE-CHECKING IF OUR FUNCTIONS INTRODUCED MISSING DATA INTO OUR NEW COLS
combined_data.isnull().sum()

text          0
class         0
text_clean    0
dtype: int64

### Length of posts
We should dive in to check out what the number of words are in an average post in each subreddit. 

In [21]:
combined_data=combined_data.reset_index()
combined_data.index

RangeIndex(start=0, stop=56, step=1)

In [22]:
combined_data["selftext_length"]= [len(combined_data["text_clean"][i]) for i in range(len(combined_data))]

In [23]:
combined_data["class"].unique()

array(['depression', 'high-suicidal', 'suicidal', 'casual-talk',
       'suicide'], dtype=object)

In [24]:
combined_data.loc[combined_data["class"] == "suicidal", "class"] = "suicide"

In [25]:
ave_length_dep_post = combined_data["selftext_length"][combined_data["class"] =="depression"].mean()
ave_length_sui_post = combined_data["selftext_length"][combined_data["class"] =="suicide"].mean()
ave_length_high_sui_post = combined_data["selftext_length"][combined_data["class"] =="high-suicidal"].mean()
ave_length_casual_post = combined_data["selftext_length"][combined_data["class"] =="casual-talk"].mean()



print("ave_length_dep_post : ",(ave_length_dep_post))
print()
print()
print("ave_length_sui_post : ", (ave_length_sui_post))
print()
print()
print ("ave_length_high_sui_post: ",ave_length_high_sui_post)
print()
print()
print("ave_length_casual_post : ",ave_length_casual_post)


ave_length_dep_post :  359.0


ave_length_sui_post :  432.125


ave_length_high_sui_post:  422.5625


ave_length_casual_post :  205.0


In [27]:
####BAaCKING UP combined_data IN A CSV FILE 
combined_data.to_csv('../data/victoria_diary_clean.csv', index = False)

In [41]:
# final_data= pd.concat([df3,df4],axis=0, ignore_index=True) 

In [49]:
# final_data.to_csv('../data/final_data_casual_combined_cleaned.csv', index = False)